In [1]:
import json
import openai
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import yfinance as yf

In [2]:
try:
    with open('API_KEY', 'r') as key_file:
        openai.api_key = key_file.read().strip()
except FileNotFoundError:
    print("API key file not found.")
except Exception as e:
    print(f"Error reading API key file: {e}")

In [82]:
openai.api_key = open('API_KEY', 'r').read()

In [1]:
def get_portfolio_tickers(risk_level, file_path):
    portfolios = load_portfolios(file_path)
    risk_level = risk_level.lower()
    if risk_level not in portfolios:
        raise ValueError("Invalid risk level. Choose from: very low, low, mid, high, very high")
    return list(portfolios[risk_level].keys())

def get_stock_price(ticker):
    return (str(yf.Ticker(ticker).history(period='1y').iloc[-1].Close)) #all return as strings 

def calculate_SMA (ticker, window): 
    data = yf.Ticker(ticker).history(period='1y').Close
    return (str(data.rolling(window=window).mean().iloc[-1]))

def calculate_EMA (ticker, window): 
    data = yf.Ticker(ticker).history(period='1y').Close
    return (str(data.ewm(span=window, adjust=False).mean().iloc[-1])) #technical analysis 

def calculate_RSI (ticker):
    data = yf.Ticker(ticker).history(period='1y').Close
    delta = data.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=14-1, adjust=False).mean()
    ema_down = down.ewm(com=14-1, adjust=False).mean()
    rs = ema_up / ema_down
    return str(100 - (100 / (1+rs)).iloc[-1])

def calculate_MACD (ticker):
    data = yf.Ticker(ticker).history(period='1y').Close
    short_EMA = data.ewm(span=12, adjust=False).mean()
    long_EMA = data.ewm(span=26, adjust=False).mean()

    MACD = short_EMA - long_EMA
    signal = MACD.ewm(span=9, adjust=False).mean()
    MACD_histogram = MACD - signal 

    return f'{MACD[-1]}, {signal[-1]}, {MACD_historgram[-1]}' #specific to chatgpt the order? 

def plot_stock_price (ticker): 
    data = yf.Ticker(ticker).history(period='1y')
    plt.figure(figsize=(10, 5))
    plt.plot( data.index, data.Close)
    plt.title(f'{ticker} Stock Price Over Last Year')
    plt.xlabel('Date')
    plt.ylabel('Stock Price ($)')
    plt.grid(True)
    plt.savefig('stock.png')
    plt.close()


# ChatGPT functions

In [76]:
functions = [
    {
    'name': 'get_portfolio_tickers',
    'description': 'Returns a list of stock tickers for the given portfolio risk level from a stored portfolio file.',
    'parameters': {
        'type': 'object',
        'properties': {
            'risk_level': {
                'type': 'string',
                'description': 'The risk level of the portfolio (very low, low, mid, high, very high).'
            },
            'file_path': {
                'type': 'string',
                'description': 'The path to the JSON file containing saved portfolio allocations.'
            }
        },
        'required': ['risk_level', 'file_path']
    }, 
    {
        'name': 'get_stock_price',
        'description': 'Gets the latest stock price given the ticker symbol of a company.',
        'parameters': {
            'type': 'object',
            'properties': {
                'ticker': {
                    'type': 'string',
                    'description': 'The stock ticker symbol for a company (for example AAPL for Apple).'
                }
            },
            'required': ['ticker']
        }
    },
    {
        'name': 'calculate_SMA',
        'description': 'Calculates the Simple Moving Average (SMA) of a stock over a given time window.',
        'parameters': {
            'type': 'object',
            'properties': {
                'ticker': {
                    'type': 'string',
                    'description': 'The stock ticker symbol for a company (e.g., AAPL for Apple).'
                },
                'window': {
                    'type': 'integer',
                    'description': 'The number of days for the moving average window.'
                }
            },
            'required': ['ticker', 'window']
        }
    },
    {
        'name': 'calculate_EMA',
        'description': 'Calculates the Exponential Moving Average (EMA) of a stock over a given time window.',
        'parameters': {
            'type': 'object',
            'properties': {
                'ticker': {
                    'type': 'string',
                    'description': 'The stock ticker symbol for a company (e.g., AAPL for Apple).'
                },
                'window': {
                    'type': 'integer',
                    'description': 'The number of days for the exponential moving average window.'
                }
            },
            'required': ['ticker', 'window']
        }
    },
    {
        'name': 'calculate_RSI',
        'description': 'Calculates the Relative Strength Index (RSI) of a stock, a momentum indicator used in technical analysis.',
        'parameters': {
            'type': 'object',
            'properties': {
                'ticker': {
                    'type': 'string',
                    'description': 'The stock ticker symbol for a company (e.g., AAPL for Apple).'
                }
            },
            'required': ['ticker']
        }
    },
    {
        'name': 'calculate_MACD',
        'description': 'Calculates the Moving Average Convergence Divergence (MACD) of a stock, a trend-following momentum indicator.',
        'parameters': {
            'type': 'object',
            'properties': {
                'ticker': {
                    'type': 'string',
                    'description': 'The stock ticker symbol for a company (e.g., AAPL for Apple).'
                }
            },
            'required': ['ticker']
        }
    },
    {
        'name': 'plot_stock_price',
        'description': 'Plots the stock price of a given ticker symbol over the last year and saves it as an image.',
        'parameters': {
            'type': 'object',
            'properties': {
                'ticker': {
                    'type': 'string',
                    'description': 'The stock ticker symbol for a company (e.g., AAPL for Apple).'
                }
            },
            'required': ['ticker']
        }
    }
]
#get good fren to do this 

In [77]:
available_functions = {
    'get_portfolio_tickers': get_portfolio_tickers
    'get_stock_price': get_stock_price,
    'calculate_SMA': calculate_SMA,
    'calculate_EMA': calculate_EMA,
    'calculate_RSI': calculate_RSI,
    'calculate_MACD': calculate_MACD,
    'plot_stock_price': plot_stock_price
}

In [81]:
if 'messages' not in st.session_state:
    st.session_state['messages'] = []

st.title('Stock Analysis Chatbot Assistant')

user_input = st.text_input('Your input:')

if user_input: 
    try:
        st.session_state['messages'].append({'role':'user', 'content': f'{user_input}'})
        response = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo-0613', #note here = 
                    messages=st.session_state['messages'],
                    functions=functions,
                    function_call='auto'
)
        response_message = response['choices'][0]['message']

        if response_message.get('function_call'):
            function_name = response_message['function_call']['name']
            function_args = json. loads(response_message['function_call']['arguments'])
            if function_name in ['get_stock_price', 'calculate_RSI', 'calculate_MACD', 'plot_stock_price']:
                args_dict = {'ticker': function_args.get('ticker')}
            elif function_name in ['calculate_SMA', 'calculate_EMA']: 
                args_dict = {'ticker': function_args.get('ticker'), 'window': function_args.get('window')}
            else:
                st.text("Function not recognized.")
                raise ValueError("Unrecognized function call.")

            function_to_call = available_function[function_name]
            function_response = function_to_call(**args_dict)

            if function_name == 'plot_stock_price': 
                st.image('stock.png')
            else: 
                st.session_state['messages'].append(response_message)
                st.session_state['messages'].append(
                    {
                    'role':'function', 
                    'name':function_name, 
                    'content': function_response

                    })
                
            second_response = openai. ChatCompletion.create(
                                        model='gpt-3.5-turbo-0613',
                                        messages=st.session_state['messages'])
            st.text(second_response['choices' ] [0] ['message' ] ['content'])
            st.session_state['messages'].append({'role':'assistant', 
                                                 'content': second_response['choices'][0]['message']['content']})
        else: 
            st.text(response_message['content'])
            st.session_state['messages'].append({'role': 'assistant', 'content': response_message['content']})
    except Exception as e:
        st.text('Error occur')
        


2024-10-31 15:20:40.463 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 15:20:40.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 15:20:40.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 15:20:40.465 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 15:20:40.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 15:20:40.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 15:20:40.467 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 15:20:40.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar